<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 9강 시계열 기반으로 데이터 집계하기

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


9강에서는 데이터의 변화를 이해하기 쉽게 표현할 수 있는 리포팅 방법과 그에 대한 내용을 소개합니다.  
우선 샘플 데이터를 불러오겠습니다. 

In [2]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.purchase_log;
CREATE TABLE sqldata.purchase_log(
    dt              string
  , order_id        integer
  , user_id         string
  , purchase_amount integer
);

INSERT INTO sqldata.purchase_log
VALUES
    ('2014-01-01',  1, 'rhwpvvitou', 13900)
  , ('2014-01-01',  2, 'hqnwoamzic', 10616)
  , ('2014-01-02',  3, 'tzlmqryunr', 21156)
  , ('2014-01-02',  4, 'wkmqqwbyai', 14893)
  , ('2014-01-03',  5, 'ciecbedwbq', 13054)
  , ('2014-01-03',  6, 'svgnbqsagx', 24384)
  , ('2014-01-03',  7, 'dfgqftdocu', 15591)
  , ('2014-01-04',  8, 'sbgqlzkvyn',  3025)
  , ('2014-01-04',  9, 'lbedmngbol', 24215)
  , ('2014-01-04', 10, 'itlvssbsgx',  2059)
  , ('2014-01-05', 11, 'jqcmmguhik',  4235)
  , ('2014-01-05', 12, 'jgotcrfeyn', 28013)
  , ('2014-01-05', 13, 'pgeojzoshx', 16008)
  , ('2014-01-06', 14, 'msjberhxnx',  1980)
  , ('2014-01-06', 15, 'tlhbolohte', 23494)
  , ('2014-01-06', 16, 'gbchhkcotf',  3966)
  , ('2014-01-07', 17, 'zfmbpvpzvu', 28159)
  , ('2014-01-07', 18, 'yauwzpaxtx',  8715)
  , ('2014-01-07', 19, 'uyqboqfgex', 10805)
  , ('2014-01-08', 20, 'hiqdkrzcpq',  3462)
  , ('2014-01-08', 21, 'zosbvlylpv', 13999)
  , ('2014-01-08', 22, 'bwfbchzgnl',  2299)
  , ('2014-01-09', 23, 'zzgauelgrt', 16475)
  , ('2014-01-09', 24, 'qrzfcwecge',  6469)
  , ('2014-01-10', 25, 'njbpsrvvcq', 16584)
  , ('2014-01-10', 26, 'cyxfgumkst', 11339)
;


""


앞선 테이블을 가지고 날짜별 매출을 집계해 보겠습니다. 

In [3]:
%%bigquery --project mygcpproject-340112
  -- 9.1 날짜별 매출과 평균 구매액을 집계하는 쿼리
SELECT
    dt
  , COUNT(*) AS purchase_count
  , SUM(purchase_amount) AS total_amount
  , AVG(purchase_amount) AS avg_amount
FROM sqldata.purchase_log
GROUP BY dt
ORDER BY dt
;

,dt,purchase_count,total_amount,avg_amount
0,2014-01-01,2,24516,12258.000000
1,2014-01-02,2,36049,18024.500000
2,2014-01-03,3,53029,17676.333333
3,2014-01-04,3,29299,9766.333333
4,2014-01-05,3,48256,16085.333333
5,2014-01-06,3,29440,9813.333333
6,2014-01-07,3,47679,15893.000000
7,2014-01-08,3,19760,6586.666667
8,2014-01-09,2,22944,11472.000000
9,2014-01-10,2,27923,13961.500000


다음으로 토, 일과 같은 주말에 매출이 변동하는지 알아보기 위해 이동평균을 사용하여 날짜별 추이를 살펴보겠습니다.

In [5]:
%%bigquery --project mygcpproject-340112
  -- 9.2 날짜별 매출과 7일 이동평균을 집계하는 쿼리
SELECT
    dt
  , SUM(purchase_amount) AS total_amount


  -- 최근 최대 7일 동안의 평균 계산하기
  , AVG(SUM(purchase_amount))
    OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
    AS seven_day_avg
  
  -- 최근 7일 동안의 평균을 확실하게 계산하기
  , CASE
      WHEN
        7 = COUNT(*)
        OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
      THEN
        AVG(SUM(purchase_amount))
        OVER(ORDER BY dt ROWS BETWEEN 6 PRECEDING AND CURRENT ROW)
    END
    AS seven_day_avg_strict
FROM sqldata.purchase_log
GROUP BY dt
ORDER BY dt
;

,dt,total_amount,seven_day_avg,seven_day_avg_strict
0,2014-01-01,24516,24516.000000,NaN
1,2014-01-02,36049,30282.500000,NaN
2,2014-01-03,53029,37864.666667,NaN
3,2014-01-04,29299,35723.250000,NaN
4,2014-01-05,48256,38229.800000,NaN
5,2014-01-06,29440,36764.833333,NaN
6,2014-01-07,47679,38324.000000,38324.000000
7,2014-01-08,19760,37644.571429,37644.571429
8,2014-01-09,22944,35772.428571,35772.428571
9,2014-01-10,27923,32185.857143,32185.857143
